In [17]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [18]:
class DayToNightDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.day_dir = os.path.join(root_dir, 'day')
        self.night_dir = os.path.join(root_dir, 'night')
        self.day_images = sorted(os.listdir(self.day_dir))
        self.night_images = sorted(os.listdir(self.night_dir))
        self.transform = transform

    def __len__(self):
        return len(self.day_images)

    def __getitem__(self, idx):
        day_path = os.path.join(self.day_dir, self.day_images[idx])
        night_path = os.path.join(self.night_dir, self.night_images[idx])
        day_img = Image.open(day_path).convert('RGB')
        night_img = Image.open(night_path).convert('RGB')

        if self.transform:
            day_img = self.transform(day_img)
            night_img = self.transform(night_img)

        return {'day': day_img, 'night': night_img}


In [19]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = DayToNightDataset(root_dir='D:\Day to Night Image Conversion using the Pix-Pix GAN\Dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:7: SyntaxWarning: invalid escape sequence '\D'
C:\Windows\Temp\ipykernel_28596\443411530.py:7: SyntaxWarning: invalid escape sequence '\D'
  dataset = DayToNightDataset(root_dir='D:\Day to Night Image Conversion using the Pix-Pix GAN\Dataset', transform=transform)


In [20]:
# Pix2Pix GAN: Day to Night Image Translation

import torch
from torch import nn

# ------------------
# Generator U-Net
# ------------------

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, transposed=False, use_dropout=False):
        super().__init__()
        if not transposed:
            self.block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )
        else:
            self.block = nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.block(x)
        return self.dropout(x) if self.use_dropout else x

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        # Encoder
        self.down1 = nn.Sequential(nn.Conv2d(in_channels, 64, 4, 2, 1), nn.LeakyReLU(0.2, inplace=True))  # No batchnorm
        self.down2 = UNetBlock(64, 128)
        self.down3 = UNetBlock(128, 256)
        self.down4 = UNetBlock(256, 512)
        self.down5 = UNetBlock(512, 512)
        self.down6 = UNetBlock(512, 512)
        self.down7 = UNetBlock(512, 512)
        self.down8 = nn.Sequential(nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU(True))  # Bottleneck

        # Decoder
        self.up1 = UNetBlock(512, 512, transposed=True, use_dropout=True)
        self.up2 = UNetBlock(1024, 512, transposed=True, use_dropout=True)
        self.up3 = UNetBlock(1024, 512, transposed=True, use_dropout=True)
        self.up4 = UNetBlock(1024, 512, transposed=True)
        self.up5 = UNetBlock(1024, 256, transposed=True)
        self.up6 = UNetBlock(512, 128, transposed=True)
        self.up7 = UNetBlock(256, 64, transposed=True)

        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        u1 = self.up1(d8)
        u2 = self.up2(torch.cat([u1, d7], dim=1))
        u3 = self.up3(torch.cat([u2, d6], dim=1))
        u4 = self.up4(torch.cat([u3, d5], dim=1))
        u5 = self.up5(torch.cat([u4, d4], dim=1))
        u6 = self.up6(torch.cat([u5, d3], dim=1))
        u7 = self.up7(torch.cat([u6, d2], dim=1))

        return self.final_up(torch.cat([u7, d1], dim=1))

# ------------------
# Discriminator
# ------------------

class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels * 2, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 1, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x, y):
        if x.size()[2:] != y.size()[2:]:
            y = nn.functional.interpolate(y, size=x.size()[2:], mode='bilinear', align_corners=False)
        return self.net(torch.cat([x, y], dim=1))


In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
G = GeneratorUNet().to(device)
D = Discriminator().to(device)

criterion_GAN = nn.BCELoss()
criterion_L1 = nn.L1Loss()

lr = 2e-4
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))


In [23]:
from torchvision.utils import save_image
epochs = 100
lambda_L1 = 100

for epoch in range(epochs):
    for i, batch in enumerate(dataloader):
        real_A = batch['day'].to(device)
        real_B = batch['night'].to(device)
        real_labels = torch.ones((real_A.size(0), 1, 30, 30)).to(device)
        fake_labels = torch.zeros((real_A.size(0), 1, 30, 30)).to(device)

        # Train Generator
        fake_B = G(real_A)
        D_fake = D(real_A, fake_B)
        loss_GAN = criterion_GAN(D_fake, real_labels)
        loss_L1 = criterion_L1(fake_B, real_B) * lambda_L1
        loss_G = loss_GAN + loss_L1

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        # Train Discriminator
        D_real = D(real_A, real_B)
        loss_D_real = criterion_GAN(D_real, real_labels)
        D_fake = D(real_A, fake_B.detach())
        loss_D_fake = criterion_GAN(D_fake, fake_labels)
        loss_D = (loss_D_real + loss_D_fake) * 0.5

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        if i % 100 == 0:
            print(f"[{epoch}/{epochs}] Batch {i} - Loss G: {loss_G.item():.4f} | Loss D: {loss_D.item():.4f}")
            save_image(fake_B * 0.5 + 0.5, f'D:\\Day to Night Image Conversion using the Pix-Pix GAN\\output/fake_{epoch}_{i}.png')


[0/100] Batch 0 - Loss G: 61.5763 | Loss D: 0.7288
[1/100] Batch 0 - Loss G: 24.4277 | Loss D: 0.0659
[2/100] Batch 0 - Loss G: 34.4151 | Loss D: 0.6994
[3/100] Batch 0 - Loss G: 13.6358 | Loss D: 0.5725
[4/100] Batch 0 - Loss G: 23.9366 | Loss D: 0.4086
[5/100] Batch 0 - Loss G: 13.4595 | Loss D: 0.7478
[6/100] Batch 0 - Loss G: 14.7603 | Loss D: 0.5091
[7/100] Batch 0 - Loss G: 12.1266 | Loss D: 0.2881
[8/100] Batch 0 - Loss G: 13.6456 | Loss D: 0.6033
[9/100] Batch 0 - Loss G: 18.2204 | Loss D: 0.3997
[10/100] Batch 0 - Loss G: 9.4045 | Loss D: 0.4152
[11/100] Batch 0 - Loss G: 18.9083 | Loss D: 0.5065
[12/100] Batch 0 - Loss G: 9.1181 | Loss D: 0.7267
[13/100] Batch 0 - Loss G: 10.3714 | Loss D: 0.4170
[14/100] Batch 0 - Loss G: 9.8503 | Loss D: 0.5132
[15/100] Batch 0 - Loss G: 10.6090 | Loss D: 0.3232
[16/100] Batch 0 - Loss G: 5.1191 | Loss D: 0.4533
[17/100] Batch 0 - Loss G: 10.4841 | Loss D: 0.6619
[18/100] Batch 0 - Loss G: 10.0072 | Loss D: 0.1794
[19/100] Batch 0 - Loss G:

In [24]:
torch.save(G.state_dict(), 'pix2pix_generator_day2night.pth')


In [25]:
import gradio as gr
import torch
from torchvision import transforms
from PIL import Image

# Define transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Load generator
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G = GeneratorUNet().to(device)
G.load_state_dict(torch.load('pix2pix_generator_day2night.pth', map_location=device))
G.eval()

# Inference function
def day_to_night(image):
    image = transform(image).unsqueeze(0).to(device)  # [1, 3, 256, 256]
    with torch.no_grad():
        fake_night = G(image)
    fake_night = (fake_night.squeeze().cpu() * 0.5 + 0.5).clamp(0, 1)  # Denormalize to [0, 1]
    fake_night_pil = transforms.ToPILImage()(fake_night)
    return fake_night_pil

# Launch Gradio app
gr.Interface(
    fn=day_to_night,
    inputs=gr.Image(type="pil", label="Upload a Day Image"),
    outputs=gr.Image(type="pil", label="Night Image"),
    title="Day to Night Image Converter 🌙",
    description="Upload a daytime image and watch it transform into night using a Pix2Pix GAN model!"
).launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
